### Table for paper, Excess Return and Tracking Err for diff paramter values on 2 index files

In [1]:
base_path="../eit_basic_experiments/exp_3" #Don't add '/' at start and end
T=200 #Training time period
C=1000000 #Capital available
file=1 #Index_file
m=8 #Inital Size of Kernel
lbuck=2 #Size Of Buckets
pho=0.1 #Transaction Cost Proportion
lamda=1/(1000*C) # lower_bound for capital invested in jth stock
nuh=0.8  # upper_bound
xii=1.2 # Limit for Tracking Error
k=14 #Cardinality constraint for stocks in portfolio
p=3 #If stock not selected in optimal soln in last p iters then it is to be dropped from kernel
f=12
#cj,cb are declared in script

In [2]:
import sys
sys.path.insert(0,"..") #Add root folder to sys path

from eit_basic_experiment import TestEitBasic
import time
from tqdm.notebook import tqdm
from IPython.utils import io
from joblib import Parallel, delayed
import pandas as pd

In [3]:
!rm -r ../eit_basic_experiments/exp_3/*

### Loop over files > Loop over paramter values and publish table
1. In sample Excess return/Tracking Err
2. Out of Sample

In [4]:
path_pattern=base_path+"/file={},pho={},xii={},k={},m={},lbuck={},p={}"

#### Make runs for parallel process

In [5]:
%%time
runs=[]
for file in [1]:
    for pho in [0.1,0.2,0.5]:
        for xii in [1.2,1.3,1.4]:
            for k in [10,16,25]:
                for m in [8,12,16]:
                    output=path_pattern.format(file,pho,xii,k,m,lbuck,p)
                    test_eit=TestEitBasic(output=output,T=T,C=C,file=file,m=m,lbuck=lbuck,\
                                        pho=pho,lamda=lamda,nuh=nuh,xii=xii,k=k,p=p,f=f)
                    runs.append(test_eit)

CPU times: user 1.06 ms, sys: 134 µs, total: 1.19 ms
Wall time: 1.25 ms


In [6]:
def parralel_func(eit_object,result_dict):
    params=eit_object.give_params()
    result_df=eit_object.run_experiment(from_root=False,verbose=False);
    result_df["z_linear"]=eit_object.objective_linear
    step_dict={}
    step_dict[params]=result_df
    result_dict.update(step_dict)
    return (result_dict)

In [7]:
%%time
result_dict={}
result_dict=Parallel(verbose = 2,n_jobs  = -1\
            )(delayed( parralel_func )( run,result_dict) for run in tqdm(runs))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.7min



CPU times: user 422 ms, sys: 99.6 ms, total: 522 ms
Wall time: 30min 25s


[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 30.4min finished


In [12]:
final_result_dict={}
for step_dict in result_dict:
    final_result_dict.update(step_dict)

In [13]:
final_result=pd.DataFrame()
for params,result_df in final_result_dict.items():
    param_part=pd.DataFrame([params])
    res_part=pd.DataFrame([result_df.loc[result_df.z_value.idxmax()]])
    temp=pd.concat([param_part,res_part.reset_index()],axis=1,ignore_index=False)
    final_result=final_result.append(temp)

In [14]:
final_result.to_csv(base_path+"/exp_3_results_table_1.csv")

In [16]:
final_result.columns

Index(['output', 'T', 'C', 'file', 'm', 'lbuck', 'pho', 'lamda', 'nuh', 'xii',
       'k', 'p', 'f', 'index', 'bucket', 'kernel_size', 'problem_status',
       'z_value', 'in_excess_return', 'in_tr_err', 'out_excess_return',
       'out_tr_err', 'z_linear'],
      dtype='object')

In [15]:
final_result.groupby(["k","pho"])[["z_value","z_linear","bucket"]].mean()

z_value     z_linear    bucket
k  pho                                    
10 0.1  1633.193709  1732.751987  0.666667
   0.2  1633.193709  1732.751987  0.666667
   0.5  1633.193709  1732.751987  0.666667
16 0.1  1729.275617  1732.751987  2.666667
   0.2  1729.275617  1732.751987  2.666667
   0.5  1729.275617  1732.751987  2.666667
25 0.1  1732.751987  1732.751987  4.000000
   0.2  1732.751987  1732.751987  4.000000
   0.5  1732.751987  1732.751987  4.000000

In [22]:
final_result.groupby(["k","m"])[["z_value","z_linear","bucket"]].mean()

z_value     z_linear    bucket
k  m                                     
10 8   1633.193709  1732.751987  2.000000
   12  1633.193709  1732.751987  0.000000
   16  1633.193709  1732.751987  0.000000
16 8   1729.275617  1732.751987  4.666667
   12  1729.275617  1732.751987  2.666667
   16  1729.275617  1732.751987  0.666667
25 8   1732.751987  1732.751987  6.000000
   12  1732.751987  1732.751987  4.000000
   16  1732.751987  1732.751987  2.000000

In [24]:
final_result.groupby(["k","xii"])[['in_excess_return', 'in_tr_err', 'out_excess_return',
       'out_tr_err',"bucket"]].mean()

in_excess_return      in_tr_err  out_excess_return    out_tr_err  \
k  xii                                                                     
10 1.2       1530.430358  224172.993279        3613.340702 -1.520349e+07   
   1.3       1637.074329  216741.760970        8291.844229 -2.024084e+07   
   1.4       1732.076441  261853.287566        8918.196357 -2.093902e+07   
16 1.2       1648.811498   85576.224460        6463.858074 -3.787245e+07   
   1.3       1729.443446  134887.449248        6857.818146 -3.817990e+07   
   1.4       1809.571907  126906.369317        7365.981180 -2.936720e+07   
25 1.2       1653.519570  106142.643581        7406.139759 -3.902740e+07   
   1.3       1733.535995  102298.539670        7258.858578 -3.885810e+07   
   1.4       1811.200395  147940.860604        7850.030115 -3.965217e+07   

          bucket  
k  xii            
10 1.2  0.666667  
   1.3  0.666667  
   1.4  0.666667  
16 1.2  3.000000  
   1.3  3.000000  
   1.4  2.000000  
25 1.2  4.000000  
   1.3  4.000000  
   1.4  4.000000